<a href="https://colab.research.google.com/github/tracy2811/24-DataScience-Projects/blob/master/BigmartSales/BigmartSales_update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Mart Sales Prediction

[Guide](https://www.analyticsvidhya.com/blog/2016/02/bigmart-sales-solution-top-20/)

## Hypothesis Generation

Understanding the problem better by brainstorming possible factors that can impact the outcome

## Data Exploration

Looking at categorical and continuous feature summaries and making inferences about the data

### Import

In [0]:
import pandas as pd, numpy as np, seaborn as sns
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/My\ Drive/Colab\ Notebooks/BigmartSales

/content/drive/My Drive/Colab Notebooks/BigmartSales


### Read files

In [4]:
train = pd.read_csv('Train_UWu5bXk.csv')
print(train.shape)

test = pd.read_csv('Test_u94Q5KV.csv')
print(test.shape)

data=pd.concat([train,test],ignore_index=True, keys=['train', 'test'])
print(data.shape)
data.head()

(8523, 12)
(5681, 11)
(14204, 12)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1
3,Regular,FDX07,182.0950,732.3800,Fruits and Vegetables,0.000000,19.20,1998,OUT010,Tier 3,NaN,Grocery Store
4,Low Fat,NCD19,53.8614,994.7052,Household,0.000000,8.93,1987,OUT013,Tier 3,High,Supermarket Type1


### Explore

In [5]:
data.isnull().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
dtype: int64

`Item_Weight` and `Outlet_Size` have missing values.

In [6]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [7]:
data.apply(lambda x: len(x.unique()))

Item_Fat_Content                 5
Item_Identifier               1559
Item_MRP                      8052
Item_Outlet_Sales             3494
Item_Type                       16
Item_Visibility              13006
Item_Weight                    416
Outlet_Establishment_Year        9
Outlet_Identifier               10
Outlet_Location_Type             3
Outlet_Size                      4
Outlet_Type                      4
dtype: int64

In [8]:
categorical_columns = [x for x in data.dtypes.index
                      if data.dtypes[x]=='object']

# exclude Id cols
categorical_columns = [x for x in categorical_columns 
                      if x not in ['Item_Identifier']]

for col in categorical_columns:
  print(data[col].value_counts())
  print()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

OUT027    1559
OUT013    1553
OUT046    1550
OUT035    1550
OUT049    1550
OUT045    1548
OUT018    1546
OUT017    1543
OUT010     925
OUT019     880
Name: Outlet_Identifier, dtype: int64

Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

Supermarket Type1

1. **Item_Fat_Content**: mis-coded `Low Fat`, `LF`, `low fat`; `Regular`, `reg`

2. **Item_Type**: not all categories have substantial numbers
  
3. **Outlet_Type**: `Supermarket Type2` and `Supermarket Type3` can be combined

## Data Cleaning

Inputing missing values in the data and checking for outliers

### Input missing values

In [9]:
data['Item_Weight'].fillna(data['Item_Weight'].mean(),inplace=True)
data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0],inplace=True)
data.isnull().sum()

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                     0
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                     0
Outlet_Type                     0
dtype: int64

## Feature Engineering

Modifying existing varibales and creating new ones for analysis

### Consider combining Outlet_Type

Check if they have similar mean sales

In [10]:
data.pivot_table(values='Item_Outlet_Sales',index='Outlet_Type')

,Item_Outlet_Sales
Outlet_Type,
Grocery Store,339.828500
Supermarket Type1,2316.181148
Supermarket Type2,1995.498739
Supermarket Type3,3694.038558


There is a significant difference between them, just leave them as it is .

### Modify Item_Visibility

The minimum value (0) which makes no practical sense will be replaced with mean visibility of that product

In [0]:
# Determine average visibility of a product
#visibility_avg = data.pivot_table(values='Item_Visibility',
#                                 index='Item_Identifier')

# Input 0 values with mean visibility of that product
#miss_bool = (data['Item_Visibility'] == 0)

#print(sum(miss_bool))

#data.loc[miss_bool,'Item_Visibility'] = \
#data.loc[miss_bool,'Item_Identifier'].apply(lambda x: visibility_avg[x])

#print(sum(data['Item_Visibility']==0))

### Create a broad category of Type of Item

In [13]:
# Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])

# Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                            'NC':'Non-Consumable',
                                                            'DR': 'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

### Determine the years of operation of a store

In [14]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

### Modify categories of Item_Fat_Content

In [15]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                            'reg':'Regular',
                                                            'low fat':'Low Fat'})
data['Item_Fat_Content'].value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [16]:
data.loc[data['Item_Type_Combined']=="None-Consumable",'Item_Fat_Content']="Non-Edible"

data['Item_Fat_Content'].value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

### Numerical and One-Hot Coding of Categorical variables

In [17]:
# One Hot Coding:
data = pd.get_dummies(data, columns=['Item_Fat_Content',
                                    'Outlet_Location_Type',
                                    'Outlet_Size', 'Outlet_Type',
                                    'Item_Type_Combined', 'Outlet_Identifier'])
data.dtypes

Item_Identifier                       object
Item_MRP                             float64
Item_Outlet_Sales                    float64
Item_Type                             object
Item_Visibility                      float64
Item_Weight                          float64
Outlet_Establishment_Year              int64
Outlet_Years                           int64
Item_Fat_Content_Low Fat               uint8
Item_Fat_Content_Regular               uint8
Outlet_Location_Type_Tier 1            uint8
Outlet_Location_Type_Tier 2            uint8
Outlet_Location_Type_Tier 3            uint8
Outlet_Size_High                       uint8
Outlet_Size_Medium                     uint8
Outlet_Size_Small                      uint8
Outlet_Type_Grocery Store              uint8
Outlet_Type_Supermarket Type1          uint8
Outlet_Type_Supermarket Type2          uint8
Outlet_Type_Supermarket Type3          uint8
Item_Type_Combined_Drinks              uint8
Item_Type_Combined_Food                uint8
Item_Type_

### Exporting data

In [23]:
data.drop(['Item_Type','Outlet_Establishment_Year'], axis=1, inplace=True)

train = data[data['Item_Outlet_Sales'].notnull()]
test = data[data['Item_Outlet_Sales'].isnull()]

test.drop(['Item_Outlet_Sales'], axis=1, inplace=True)

train.to_csv("train_modified.csv", index=False)
test.to_csv("test_modified.csv", index=False)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


## Model Building

Making predictive models on the data

### Baseline model

In [25]:
mean_sales = train['Item_Outlet_Sales'].mean()

base1 = test[['Item_Identifier']]
base1['Item_Outlet_Sales'] = mean_sales

base1.to_csv('alg0.csv',index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


### Linear Regression Model

### Ridge Regression Model

### Decision Tree Model

### Random Forest Model